# 使用Python构建NLP预训练基础-Vocabulary

在处理NLP任务时，通常来说，文本数据预的处理流程大致如下：

1. 获取原始文本簇 Corpus
2. 处理文本
3. 将文本标签化处理
4. 生成文本簇字典 Vocabulary
5. 处理文本表示

而构建字典的过程是在文本表示和下游任务之前必须进行的操作

构建字典的过程并不仅仅是将文本簇中的文字以Set的方式存储，还可以存储文本簇中的元数据信息


## 第一步

创建用于划定文本范围的字符

1. start of sentence 标识句子的开头
2. end of sentence 标识句子的结尾
3. padding of sentence 填充长度较短的文本

In [1]:
PAD_token = 0   # Used for padding short sentences
SOS_token = 1   # Start-of-sentence token
EOS_token = 2   # End-of-sentence token

## 第二步

- 创建一个 Vocabulary 类的构造函数
- 存储文本簇的元数据信息

In [2]:
def __init__(self, name):
  self.name = name
  self.word2index = {} # 单词转下标的字典
  self.word2count = {} # 单词计数的字典
  self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"} # 下标转单词的字典
  self.num_words = 3 # 字典中的单词总数
  self.num_sentences = 0 # 文本簇中的句子总数
  self.longest_sentence = 0 # 文本簇中最长的句子

## 第三步

- 创建填充字典的函数，以单词为维度

In [4]:
def add_word(self, word): # 以单词为输入维度，将单词送入字典中存储
  # 基于字典已经存储过当前单词和未存储过当前单词作为分界
  # 按照单词送入类的顺序进行下标计数，没有特殊的处理步骤
  if word not in self.word2index:
    # First entry of word into vocabulary
    self.word2index[word] = self.num_words
    self.word2count[word] = 1
    self.index2word[self.num_words] = word
    self.num_words += 1
  else:
    # 当单词已经存在于字典中时，直接对计数+1
    self.word2count[word] += 1

- 创建填充字典的函数，以句子为维度，使用刚刚创建的单词创建函数
- 这里对于文本的处理仅有，使用空格进行分词的操作，是偷懒的做法，真实的场景会更加复杂

In [5]:
def add_sentence(self, sentence): # 以句子为输入维度
  sentence_len = 0
  for word in sentence.split(' '):
    sentence_len += 1
    self.add_word(word)
  if sentence_len > self.longest_sentence:
    # 每一次输入的新句子，若长度超过之前最长的句子，则更新最长句子的原信息
    self.longest_sentence = sentence_len
  # 计算文本簇中句子的数量
  self.num_sentences += 1

- 创建两个查找函数，扩充字典类的基本查询功能

1. 基于单词查询下标
2. 基于下标查询单词

- 将上述内容串联成 Vocabulary 类

In [6]:
class Vocabulary:
    PAD_token = 0
    SOS_token = 1   
    EOS_token = 2   

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence.split(' '):
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            self.longest_sentence = sentence_len
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

## 使用简单文本构造一个字典

In [7]:
voc = Vocabulary('test')
print(voc)

In [8]:
# 假定一个简单的文本簇
corpus = ['This is the first sentence.',
          'This is the second.',
          'There is no sentence in this corpus longer than this one.',
          'I love China.']
print(corpus)

['This is the first sentence.', 'This is the second.', 'There is no sentence in this corpus longer than this one.', 'I love China.']


In [9]:
for sent in corpus:
  voc.add_sentence(sent)

print('Token 4 corresponds to token:', voc.to_word(4))
print('Token "this" corresponds to index:', voc.to_index('this'))

Token 4 corresponds to token: is
Token "this" corresponds to index: 13


In [10]:
# 遍历整个字典

for word in range(voc.num_words):
    print(voc.to_word(word))

PAD
SOS
EOS
This
is
the
first
sentence.
second.
There
no
sentence
in
this
corpus
longer
than
one.
I
love
China.


In [11]:
# 将文本簇中的第三个句子，转化为字典index的形式
# 在转化之前，我们需要对句子的开头结尾加上SOS和EOS

sent_tkns = []
sent_idxs = []
for word in corpus[3].split(' '):
  sent_tkns.append(word)
  sent_idxs.append(voc.to_index(word))
print(sent_tkns)
print(sent_idxs)

['I', 'love', 'China.']
[18, 19, 20]


- 虽然这样一个最base的 Vocabulary 已经实现了，但是细节上仍然还有很多缺陷，比如：

1. 没有对文本进行标准化处理（大小写归一，停用词，标点符号）
2. 虽然使用了 SOS 和 EOS 但是没有对句子的长度进行规划，没有使用 padding symbol
3. 没有对字典表进行修剪（对于出现频率极少的词进行适当的剔除操作，降低字典表的大小）